# Cit Citas

In [1]:
# Cargar paquetes
from datetime import datetime
import pandas as pd
from sqlalchemy import and_, or_
from sqlalchemy.sql import func
from lib.database import SessionLocal
db = SessionLocal()

In [2]:
# Cargar modelos
from citas_admin.v2.autoridades.models import Autoridad
from citas_admin.v2.cit_categorias.models import CitCategoria
from citas_admin.v2.cit_citas.models import CitCita
from citas_admin.v2.cit_clientes.models import CitCliente
from citas_admin.v2.cit_clientes_recuperaciones.models import CitClienteRecuperacion
from citas_admin.v2.cit_clientes_registros.models import CitClienteRegistro
from citas_admin.v2.cit_dias_inhabiles.models import CitDiaInhabil
from citas_admin.v2.cit_horas_bloqueadas.models import CitHoraBloqueada
from citas_admin.v2.cit_oficinas_servicios.models import CitOficinaServicio
from citas_admin.v2.cit_servicios.models import CitServicio
from citas_admin.v2.distritos.models import Distrito
from citas_admin.v2.domicilios.models import Domicilio
from citas_admin.v2.materias.models import Materia
from citas_admin.v2.modulos.models import Modulo
from citas_admin.v2.oficinas.models import Oficina
from citas_admin.v2.permisos.models import Permiso
from citas_admin.v2.roles.models import Rol
from citas_admin.v2.usuarios.models import Usuario
from citas_admin.v2.usuarios_roles.models import UsuarioRol

In [3]:
# Calcular cantidades de citas por oficina y servicio
inicio_desde = datetime(2022, 8, 1, 0, 0, 0)
inicio_hasta = datetime(2022, 8, 5, 23, 59, 59)
matriz = db.\
    query(
        Oficina.clave.label("oficina"),
        CitServicio.clave.label("servicio"),
        func.count("*").label("cantidad"),
    ).\
    select_from(CitCita).join(CitServicio, Oficina).\
    filter(CitCita.estatus == "A").\
    filter(or_(CitCita.estado == "ASISTIO", CitCita.estado == "PENDIENTE")).\
    filter(CitCita.inicio >= inicio_desde).\
    filter(CitCita.inicio <= inicio_hasta).\
    filter(CitServicio.estatus == "A").\
    filter(Oficina.estatus == "A").\
    group_by(Oficina.clave, CitServicio.clave)
print(matriz)

SELECT oficinas.clave AS oficina, cit_servicios.clave AS servicio, count(%(count_1)s) AS cantidad 
FROM cit_citas JOIN cit_servicios ON cit_servicios.id = cit_citas.cit_servicio_id JOIN oficinas ON oficinas.id = cit_citas.oficina_id 
WHERE cit_citas.estatus = %(estatus_1)s AND (cit_citas.estado = %(estado_1)s OR cit_citas.estado = %(estado_2)s) AND cit_citas.inicio >= %(inicio_1)s AND cit_citas.inicio <= %(inicio_2)s AND cit_servicios.estatus = %(estatus_2)s AND oficinas.estatus = %(estatus_3)s GROUP BY oficinas.clave, cit_servicios.clave


In [4]:
# Create dataframe from query
df = pd.read_sql(matriz.statement, db.bind)
df.oficina = df.oficina.astype("category")
df.servicio = df.servicio.astype("category")
# Show info
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   oficina   190 non-null    category
 1   servicio  190 non-null    category
 2   cantidad  190 non-null    int64   
dtypes: category(2), int64(1)
memory usage: 4.9 KB
None


In [5]:
# Create pivot table
pivot = pd.pivot_table(df, index="oficina", columns="servicio", values="cantidad", aggfunc=sum)
print(pivot)

servicio         CHECER  CITAACT  CITJUE  DEVDOC  ENTDEMANE  EXPCOP  EXPCS  \
oficina                                                                      
ACN-J-M-CIV-FAM       0        2       2       1          0       0      0   
ACN-J-PEN-O           0        0       0       0          0       0      0   
DACU35-01JF           1        3       0       0          0       0      0   
DACU35-AREG           0        0       0       0          0       0      0   
DMON25-01JC           0        1       0       0          0       0      0   
DMON25-01JF           0        0       0       0          0       0      0   
DMON25-02JC           0        0       0       0          0       0      0   
DMON25-02JF           1        2       1       0          0       0      0   
DMON25-03JC           0        1       0       0          0       0      0   
DMON25-03JF           0        2       0       0          0       0      0   
DMON25-04JF           0        1       1       0          0     